# Analise de sentimento

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk

##  Read csv file

In [2]:
df = pd.read_csv('./datasets/sentimentos.csv')

## View Data

In [3]:
df.sample(10)

,Texto,Sentimento
437,uau que noticia bacana,surpresa
685,esta comida me parece muito atraente,alegria
780,e abominável o que fazem com os animais,medo
606,estou sem dinheiro para a comida,tristeza
851,que bela surpresa você me fez,surpresa
298,"eu te entrego o dinheiro, por favor não me mate!",medo
154,suas bobagens estão nos incomodando,nojo
487,estamos seguindo o caminho errado!,surpresa
498,isso e realmente chocante,surpresa
627,só me resta a esperança,tristeza


In [4]:
df['Sentimento'].value_counts()

alegria     162
medo        156
raiva       152
tristeza    149
nojo        143
surpresa    141
Name: Sentimento, dtype: int64

In [35]:
df_alegria = df.loc[df.Sentimento == 'alegria'][0:140]
df_medo = df.loc[df.Sentimento == 'medo'][0:140]
df_raiva = df.loc[df.Sentimento == 'raiva'][0:140]
df_tristeza = df.loc[df.Sentimento == 'tristeza'][0:140]
df_nojo = df.loc[df.Sentimento == 'nojo'][0:140]
df_surpresa = df.loc[df.Sentimento == 'surpresa'][0:140]

Texto         140
Sentimento    140
Palavras      140
dtype: int64

## Load stop words 

In [5]:
stop_words = nltk.corpus.stopwords.words('portuguese')

## Stemming: Removing sulfix and prefix of word

In [6]:
def apply_stemmer(text: str):
    stemmer = nltk.stem.RSLPStemmer()
    texts = []
    texts = [str(stemmer.stem(word)) for word in text.split() if word not in stop_words]
    return tuple(texts)

In [7]:
apply_stemmer('Ola admirável mundo novo, eu programei você!! Eu sou um programador')

('ola', 'admir', 'mund', 'novo,', 'program', 'você!!', 'eu', 'program')

In [8]:
df['Palavras'] = df['Texto'].apply(apply_stemmer)
df.sample(10)

,Texto,Sentimento,Palavras
193,que criança mal educada,nojo,"(crianç, mal, educ)"
741,vamos alarmar a todos sobre a situação,medo,"(vam, alarm, tod, sobr, situ)"
605,ele não gostou da minha comida,tristeza,"(gost, com)"
413,eu faco dieta e constinuo engordando,raiva,"(fac, diet, constinu, engord)"
778,fui notificado e estou com medo de perde a gua...,medo,"(notific, med, perd, guard, filh)"
539,não me deixe sozinha,tristeza,"(deix, so)"
375,seu descaso e frustrante,raiva,"(descas, frustr)"
552,acabaram minhas ferias,tristeza,"(acab, fer)"
501,"nossa, que top esse filme!, eu nao esperava",surpresa,"(nossa,, top, filme!,, nao, esper)"
168,esse perfume e enjoativo,nojo,"(perfum, enjo)"


## Getting words

In [9]:
words = []
for index, item in df.iterrows():
    words.extend(item.Palavras)

In [10]:
len(words)

2797

## Genereting the frequency distribution

In [11]:
frequency = nltk.FreqDist(words)
frequency.most_common(10)

[('vou', 32),
 ('nao', 30),
 ('quer', 23),
 ('tão', 22),
 ('tod', 20),
 ('ser', 20),
 ('sint', 19),
 ('tud', 19),
 ('pod', 18),
 ('fic', 17)]

## Getting unique words

In [12]:
unique_words = frequency.keys()
print(len(unique_words))

1290


In [13]:
def extract_words_to_text(text):
    doc = None
    if(type(text) == type(str)):
        doc = text.split()
    else:
        doc = set(text)
        
    features = {}
    for word in unique_words:
        features['%s' % word] = (word in doc)
    return features

In [14]:
def extract_data(df: pd.DataFrame):
    itens = []
    for i, row in df.iterrows():
        field = (tuple(row['Palavras']), row['Sentimento'])
        itens.append(field)
    return itens

In [15]:
extract_data(df)[0:5]

[(('trabalh', 'agrad'), 'alegria'),
 (('gost', 'fic', 'aconcheg'), 'alegria'),
 (('fiz', 'ades', 'curs', 'hoj', 'porqu', 'gost'), 'alegria'),
 (('admir', 'muit'), 'alegria'),
 (('ador',), 'alegria')]

## Splitting the data frame into train and test

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['Texto', 'Palavras']], df['Sentimento'], train_size=0.85, random_state=42)

In [17]:
y_train.value_counts()

alegria     140
raiva       133
medo        127
nojo        123
surpresa    122
tristeza    122
Name: Sentimento, dtype: int64

In [18]:
y_test.value_counts()

medo        29
tristeza    27
alegria     22
nojo        20
surpresa    19
raiva       19
Name: Sentimento, dtype: int64

In [19]:
X_train['Sentimento'] = y_train

## Train model

In [20]:
data = nltk.classify.apply_features(extract_words_to_text, extract_data(X_train), labeled=True)

In [21]:
classificador = nltk.NaiveBayesClassifier.train(data)

In [22]:
classificador.labels()

['nojo', 'tristeza', 'medo', 'raiva', 'alegria', 'surpresa']

In [23]:
classificador.show_most_informative_features(10)

Most Informative Features
                     med = True             medo : alegri =      9.9 : 1.0
                     vou = True            raiva : triste =      8.9 : 1.0
                    real = True           surpre : alegri =      5.7 : 1.0
                     ser = True             medo : raiva  =      5.2 : 1.0
                 depress = True           triste : medo   =      5.2 : 1.0
                      am = True           alegri : nojo   =      5.0 : 1.0
                    pass = True            raiva : alegri =      4.6 : 1.0
                    sint = True           triste : medo   =      4.5 : 1.0
                   notic = True           surpre : alegri =      4.2 : 1.0
                     vid = True           triste : alegri =      4.2 : 1.0


## Evalueting model

In [24]:
print(nltk.classify.accuracy(classificador, data))

0.9452411994784876


In [25]:
def error_metric(data) -> []:
    erros = []
    for (words, target) in data:
        result = classificador.classify(words)
        if result != target:
            erros.append((target, result, words))
    print(f'Erros: {(round(len(erros) / len(data) * 100,2))} %')

In [26]:
error_metric(data)

Erros: 5.48 %


In [27]:
from nltk.metrics import ConfusionMatrix

def view_confusion_matrix(data):
    y = []
    pred = []
    for (word, target) in data:
        result = classificador.classify(word)
        y.append(target)
        pred.append(result)

    matriz = ConfusionMatrix(y, pred)
    print(matriz)

In [28]:
view_confusion_matrix(data)

         |                   s   t |
         |   a               u   r |
         |   l               r   i |
         |   e           r   p   s |
         |   g   m   n   a   r   t |
         |   r   e   o   i   e   e |
         |   i   d   j   v   s   z |
         |   a   o   o   a   a   a |
---------+-------------------------+
 alegria |<139>  .   .   1   .   . |
    medo |   8<112>  1   4   1   1 |
    nojo |   1   .<121>  .   1   . |
   raiva |   5   .   2<124>  1   1 |
surpresa |   4   .   .   1<116>  1 |
tristeza |   3   .   1   4   1<113>|
---------+-------------------------+
(row = reference; col = test)



## Teste model

In [29]:
X_test['Sentimento'] = y_test
data_test = nltk.classify.apply_features(extract_words_to_text, extract_data(X_test), labeled=True)

In [30]:
print(nltk.classify.accuracy(classificador, data_test))

0.38235294117647056


In [31]:
error_metric(data_test)

Erros: 61.76 %


In [32]:
view_confusion_matrix(data_test)

         |              s  t |
         |  a           u  r |
         |  l           r  i |
         |  e        r  p  s |
         |  g  m  n  a  r  t |
         |  r  e  o  i  e  e |
         |  i  d  j  v  s  z |
         |  a  o  o  a  a  a |
---------+-------------------+
 alegria |<14> 1  .  3  2  2 |
    medo | 13 <3> 2  3  5  3 |
    nojo |  1  3 <9> 4  2  1 |
   raiva |  9  .  . <4> .  6 |
surpresa |  5  1  1  2<10> . |
tristeza |  4  2  2  5  2<12>|
---------+-------------------+
(row = reference; col = test)



In [33]:
test = 'Que merda, não passei.'
test = apply_stemmer(test)
novo = extract_words_to_text(test)
distribuicao = classificador.prob_classify(novo)
for clas in distribuicao.samples():
    print('%s: %f' % (clas, distribuicao.prob(clas)))


nojo: 0.134556
tristeza: 0.129054
medo: 0.080013
raiva: 0.176868
alegria: 0.356048
surpresa: 0.123461


In [34]:
y = ('um', 'dois')
set(y)

{'dois', 'um'}